<a href="https://www.kaggle.com/code/sahidvelji/cleaning-the-ontario-sunshine-list-data?scriptVersionId=100028357" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Cleaning The Ontario Sunshine List Data

<a id="table-of-contents"></a>

This notebook was used to clean Ontario's public sector salary disclosure data (also known as the Ontario sunshine list). The cleaned dataset can be found [here](https://www.kaggle.com/sahidvelji/the-ontario-sunshine-list), and an EDA of the 2019 data can be found [here](https://www.kaggle.com/sahidvelji/the-ontario-sunshine-list-2019-eda).

## Table of contents

<p style="line-height: 1.6em;">
    <a href="#loading-data">1. Loading the data</a><br>
    <a href="#renaming-columns">2. Renaming columns</a><br>
    <a href="#data-types">3. Data types</a><br>
    <a href="#missing-values">4. Missing values</a><br>
    <a href="#concat-data">5. Concatenating the dataframes</a><br>
    <a href="#write-to-csv">6. Writing to CSV</a><br>
</p>

In [1]:
import os
import re
import pandas as pd
import numpy as np
import shutil
from IPython.display import display, HTML
import plotly.express as px
from pathlib import Path

YEARS = range(1996, 2020)

px.defaults.template = 'plotly_white'
px.defaults.color_discrete_sequence = ['steelblue']
MODE_BAR_BUTTONS = ['toImage', 'zoom2d', 'pan2d', 'select2d', 'lasso2d',
                    'zoomIn2d', 'zoomOut2d', 'autoScale2d', 'resetScale2d',
                    'toggleSpikelines', 'hoverClosestCartesian', 'hoverCompareCartesian']
CONFIG = {
    'modeBarButtonsToRemove': ['pan2d', 'select2d', 'lasso2d', 'toggleSpikelines']
}

# Loading the data

In [2]:
path = Path('/kaggle/input/the-ontario-sunshine-list-raw-data')  
filenames = os.listdir(path)
filenames

['en-2018-pssd-compendium.csv',
 'en-2003-pssd.csv',
 'en-2004-pssd.csv',
 'en-2016-pssd-compendium-20171128-utf8.csv',
 'en-2006-pssd.csv',
 'en-1999-pssd.csv',
 'en-2018-pssd-compendium-20191223.csv',
 'en-2002-pssd.csv',
 'en-2008-pssd.csv',
 'pssd-en-2013.csv',
 'en-2000-pssd.csv',
 'en-2007-pssd.csv',
 'en-2009-pssd.csv',
 'en-1997-pssd.csv',
 '2010.csv',
 'en-2005-pssd.csv',
 'tbs-pssd-compendium-en-utf8-2019.csv',
 'en-1998-pssd.csv',
 '2014-pssd-full-compendium-utf8-en.csv',
 'en-2001-pssd.csv',
 '2011_0.csv',
 'en-1996-pssd.csv',
 'pssd-en-2012-fixed_0.csv',
 'en-2015-pssd-compendium-with-addendum-20161219.csv']

While downloading the data from the government of Ontario website, I realized that the filenames were inconsistent. Even worse, there are two files that appear to both be data from 2018: `en-2018-pss-compendium.csv` and `en-2018-pss-compendium-20191223.csv`. Let's examine these two files.

In [3]:
pd.read_csv(path/'en-2018-pssd-compendium.csv', nrows=5)

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
0,Colleges,Abiscott,Alexa,"$197,073.24","$2,033.69",Sheridan College Institute of Technology and A...,General Counsel And Information Privacy Officer,2017
1,Colleges,Ahn,Song Ho,"$114,331.63",$51.57,Sheridan College Institute of Technology and A...,Visualization Researcher - Part-Time Faculty,2017
2,Colleges,Aitken,Sharon,"$118,650.76","$1,250.00",Sheridan College Institute of Technology and A...,Director Development and Campaign,2017
3,Colleges,Ali,Shirook,"$104,398.32",$101.50,Sheridan College Institute of Technology and A...,Professor,2017
4,Colleges,Allcott,Austin Micha,"$124,307.70","$1,264.54",Sheridan College Institute of Technology and A...,Dean,2017


In [4]:
pd.read_csv(path/'en-2018-pssd-compendium-20191223.csv', nrows=5)

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
0,Colleges,Jensen,Cheryl,"$ 337,533.24","$ 3,408.12",Algonquin College,President,2018
1,Colleges,Brule,Claude,"$ 230,976.76",$ 374.96,Algonquin College,Senior Vice President Academic,2018
2,Colleges,Wotherspoon,Doug,"$ 230,788.38",$ 186.58,Algonquin College,Vice President Innovation and Strategy,2018
3,Colleges,Mcnair,Duane,"$ 221,744.64",$ 366.12,Algonquin College,Vice President Finance and Administration,2018
4,Colleges,Stanbra,Laura,"$ 221,744.64",$ 366.12,Algonquin College,Vice President Student Services,2018


It turns out that `en-2018-pss-compendium.csv` is actually data from 2017 and `en-2018-pss-compendium-20191223.csv` is data from 2018. The input directory is read-only data, so we cannot rename files here. However, we will use a dictionary to organize the files by year after loading them into dataframes. First, we'll create a regular expression to extract the year from each filename. 

In [5]:
p = re.compile('(?:^|-)(\d{4})\D')

That should do it. The year appears either at the beginning of the filename or after a dash. The character following the year is either a dash, a dot, or an underscore. This means we have a non-numeric character following the year. The regular expression was constructed based on these observations: first we match the beginning of a string or a dash in a non-capturing group. Then, we match 4 digits in a capturing group. This is followed by a single non-numeric digit.

Unfortunately, we encounter a `UnicodeDecodeError` if we try to load the data with the standard utf-8 encoding for some of the files. Instead, we'll use latin1 encoding if the default fails. The error occurs due to characters such as "é" in the data.

In [6]:
def read_csv(filename):
    try:
        return pd.read_csv(path/filename, encoding='utf-8')
    except UnicodeDecodeError:
        return pd.read_csv(path/filename, encoding='latin1')

In [7]:
pss = {}
for filename in filenames:
    if filename == 'en-2018-pssd-compendium.csv':
        pss[2017] = read_csv(filename)
        continue
    m = p.search(filename)
    year = int(m.group(1))
    pss[year] = read_csv(filename)

Now we have a dictionary where each key is a calendar year and the value is the corresponding dataframe. The `describe` method is a useful way of displaying a summary table of the data. Examining all 24 tables one by one wouldn't be very efficient but we'll display the tables here for reference purposes anyways.

In [8]:
for year in YEARS:
    display(HTML(pss[year].describe(include='all').to_html()))

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year,Unnamed: 8
count,4501,4501,4500,4501,4501,4501,4501,4501.0,0.0
unique,8,3627,2453,3544,2032,441,1942,NaN,NaN
top,UNIVERSITIES,Brown,John,"$123,825.00",$0.00,Attorney General,"Provincial Judge, OCJ",NaN,NaN
freq,1190,16,82,224,304,624,225,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996.0,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996.0,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996.0,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996.0,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996.0,NaN


,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
count,5378,5378,5378,5378,5378,5378,5377,5378.0
unique,9,4189,2930,4502,3278,467,2290,NaN
top,Universities,Smith,John,"$127,779.36",$0.00,Ontario Hydro,Prof.,NaN
freq,1439,24,97,206,216,779,251,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997.0
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997.0
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997.0
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1997.0


,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
count,6296,6296,6295,6296,6296,6296,6296,6296.0
unique,8,4416,2737,5261,3415,441,2528,NaN
top,Crown Agencies,SMITH,JOHN,"$129,716.86",$0.00,Ontario Hydro,Professor,NaN
freq,1818,38,120,221,378,1412,256,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1998.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1998.0
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1998.0
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1998.0
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1998.0


,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
count,8125,8125,8125,8125,8125,8125,8125,8125.0
unique,9,5484,3430,7112,4697,481,3192,NaN
top,Hydro One & Ontario Power Generation,SMITH,JOHN,"$173,478.68",$0.00,Ontario Power Generation,Professor,NaN
freq,2153,48,235,202,489,1759,364,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.0
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.0
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.0
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.0


,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
count,10352,10352,10352,10352,10352,10352,10352,10352.0
unique,9,6677,4034,9264,5544,499,4081,NaN
top,Hydro One & Ontario Power Generation,SMITH,JOHN,"$168,318.24",$0.00,Ontario Power Generation,Professor,NaN
freq,3103,58,285,222,595,2633,615,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.0
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.0
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.0
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.0


...

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
count,115920,115920,115920,115920,115920,115920,115920,115920.000000
unique,18,44759,20504,98539,36262,1670,26401,NaN
top,Municipalities & Services,Smith,David,"$207,080.29",$-,Ontario Power Generation,Professor,NaN
freq,34375,672,2220,328,6980,7632,5289,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015.000104
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.013776
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015.000000


,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
count,124445,124445,124445,124445,124444,124445,124445,124445
unique,17,47797,22708,106860,39226,1788,27017,38
top,Municipalities and Services,Smith,Michael,"$100,252.95",$0.00,Ontario Power Generation,Professor,2016
freq,35684,707,2280,543,7653,7730,5861,124380


,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
count,131909,131909,131909,131909,131909,131909,131909,131909.0
unique,18,49656,25378,113341,42967,1857,29286,NaN
top,Municipalities and Services,Smith,Michael,"$101,974.84",$0.00,Ontario Power Generation,Constable,NaN
freq,40798,729,2376,750,8704,7878,4040,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017.0
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017.0
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017.0
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017.0


,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
count,151375,151375,151375,151375,151375,151375,151375,151375.0
unique,17,55226,27054,128976,44908,1904,30992,NaN
top,Municipalities and Services,Smith,Michael,"$ 103,588.16",$ 0.00,Ontario Power Generation,Professor,NaN
freq,44469,817,2716,912,16703,7919,7180,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018.0
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018.0
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018.0
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018.0


,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
count,166977,166977,166977,166977,166977,166977,166977,166977.0
unique,17,59590,27006,139760,46108,1898,32868,NaN
top,Municipalities & Services,Smith,Michael,"$104,542.37",$0.00,Ontario Power Generation,Professor,NaN
freq,48321,900,3029,1112,25396,8043,7094,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.0
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.0
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.0
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019.0


The Salary Paid and Taxable Benefits columns should have type `float` but it looks like they are currently strings because of the dollar signs and commas. We will fix this later.

First, we will make sure that the column names of all dataframes match.

Second, we will deal with the data types of the columns. As mentioned above, we expect the Salary Paid and Taxable Benefits columns to have type `float`. The Calendar Year column should have type `int` and the rest of the columns should have type `string`.

Then, we will check for missing values and see if there is a good way of imputing them. Finally, we will explore the possibility of concatenating the dataframes. After all, they should have the same number of columns, the same column names and data types, and one dataframe is easier to work with than 24 separate ones.

<a id="renaming-columns"></a>
[Return to table of contents](#table-of-contents)

# Renaming columns

We will check to make sure that every dataframe has the same column names using the 2019 columns as a reference.

In [9]:
refcols = pss[2019].columns
for year in YEARS:
    if not refcols.equals(pss[year].columns):
        print(year, pss[year].columns.tolist(), sep='\n', end='\n\n')

print("2019", refcols.tolist(), sep='\n')

1996
['Sector', 'Last Name', 'First Name', 'Salary Paid', 'Taxable Benefits', 'Employer', 'Job Title', 'Calendar Year', 'Unnamed: 8']

2001
['Sector', 'Surname', 'First Name', 'Salary Paid', 'Taxable Benefits', 'Employer', 'Position', 'Calendar Year']

2009
['Sector', 'Last Name', 'First Name', 'Salary Paid ', 'Taxable Benefits', 'Employer', 'Job Title', 'Calendar Year']

2010
['Sector', 'Last Name', 'First Name', 'Salary Paid ', 'Taxable Benefits', 'Employer', 'Job Title', 'Calendar Year']

2011
['Sector', 'Last Name', 'First Name', 'Salary Paid ', 'Taxable Benefits', 'Employer', 'Job Title', 'Calendar Year']

2014
['Sector', 'Last name', 'First Name', 'Salary Paid', 'Taxable Benefits', 'Employer', 'Job title', 'Calendar year']

2019
['Sector', 'Last Name', 'First Name', 'Salary Paid', 'Taxable Benefits', 'Employer', 'Job Title', 'Calendar Year']


There are numerous inconsistencies:

- There is an extra column in the 1996 dataframe. 
- The 2001 dataframe has Surname instead of Last Name and Position instead of Job Title. 
- There are trailing whitespaces in the Salary Paid column in the 2009, 2010, and 2011 dataframes.
- In the 2014 dataframe, the second words of "Last name", "Job title", and "Calendar year" are not capitalized. 

We will first examine the extra column in the 1996 dataframe and drop it if appropriate.

In [10]:
pss[1996].head()

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year,Unnamed: 8
0,OTHER PUBLIC SECTOR EMPLOYERS,Kendall,Perry,"$194,890.40",$711.24,Addiction Research Foundation,President & CEO,1996,NaN
1,OTHER PUBLIC SECTOR EMPLOYERS,Rehm,Juergen,"$115,603.62",$403.41,Addiction Research Foundation,"Dir., Soc. Eval. Research & Act. Dir., Clin. R...",1996,NaN
2,OTHER PUBLIC SECTOR EMPLOYERS,Room,Robin,"$149,434.48",$512.58,Addiction Research Foundation,"V.P., Research & Coordinator, Intern. Programs",1996,NaN
3,ONTARIO PUBLIC SERVICE,KNOX,KEN W,"$109,382.92","$4,921.68","Agriculture,Food and Rural Affairs",Deputy Minister,1996,NaN
4,HOSPITALS,Cliff,Bruce,"$110,309.00","$3,157.00",Ajax and Pickering General Hospital,President & CEO,1996,NaN


In [11]:
pss[1996]['Unnamed: 8'].isna().all()

True

This unnamed column is filled with `NaN` values, meaning that we can safely drop this column.

In [12]:
pss[1996] = pss[1996].drop(columns='Unnamed: 8')

Now that every dataframe has the same number of columns, let's rename all columns to match the columns of the 2019 dataframe.

In [13]:
for year in YEARS:
    pss[year].columns = refcols

The Calendar Year column may seem redundant because of the file name. For example, we know that all of the data in `tbs-pssd-compendium-en-utf8-2019.csv` is for the calendar year 2019. However, we will not drop this column yet in case we later want to concatenate the dataframes.

<a id="data-types"></a>
[Return to table of contents](#table-of-contents)

# Data types

Here, we will examine the data types of the columns and ensure that they match across all dataframes.

In [14]:
for year in YEARS:
    print(year, pss[year].dtypes, sep='\n', end='\n\n')

1996
Sector              object
Last Name           object
First Name          object
Salary Paid         object
Taxable Benefits    object
Employer            object
Job Title           object
Calendar Year        int64
dtype: object

1997
Sector              object
Last Name           object
First Name          object
Salary Paid         object
Taxable Benefits    object
Employer            object
Job Title           object
Calendar Year        int64
dtype: object

1998
Sector              object
Last Name           object
First Name          object
Salary Paid         object
Taxable Benefits    object
Employer            object
Job Title           object
Calendar Year        int64
dtype: object

1999
Sector              object
Last Name           object
First Name          object
Salary Paid         object
Taxable Benefits    object
Employer            object
Job Title           object
Calendar Year        int64
dtype: object

2000
Sector              object
Last Name           obje

Unfortunately, we discover more inconsistencies. Quickly scrolling through the output tells us that for most dataframes, the Calendar Year column has data type `int`, except for the 2016 dataframe. Also, the dataframes for 2012 and 2013 have type `float` for the Salary Paid and Taxable Benefits columns while the other dataframes have type `object` for these columns.

Let's find out why the Calendar Year column in the 2016 dataframe doesn't have type `int`.

In [15]:
pss[2016]['Calendar Year'].nunique()

38

That doesn't look promising. We expect a single unique value for the Calendar Year column, but we have 38 unique values instead.

In [16]:
wrong_year = pss[2016][pss[2016]['Calendar Year'] != '2016']
wrong_year.shape[0]

65

In [17]:
wrong_year.head(20)

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
6223,Crown Agencies,Fagan,Thomas,"$126,498.77",$0.00,Landlord and Tenant Board,Social Benefits Tribunal,Member (Part-time) / Membre (à temps partiel)
9323,Crown Agencies,Doran,Patrick,"$121,415.63",$0.00,Social Benefits Tribuna,Child and Family Services Review Board and Cu...,Member (Part-time) / Membre (à temps partiel)
23040,Government of Ontario - Ministries,LeBlanc,Laurie,"$242,431.56","$23,569.27",Municipal Affairs,Housing,Deputy Minister
32225,Hospitals and Boards of Public Health,Levac,Jody J.,"$169,367.20",$622.35,Southlake Regional Health Centre,Director,seconded to Stevenson Memorial Hospital as CEO
32238,Hospitals and Boards of Public Health,Mccarron,Veronica J.,"$133,124.22",$493.25,Southlake Regional Health Centre,Manager,seconded to Cancer Care Ontario
32319,Hospitals and Boards of Public Health,Wells,Antoinette L.,"$167,927.24",$0.00,Southlake Regional Health Centre,Director,resourced to Federal Development Grant
110271,Universities,Phillips,Paulette,"$130,795.50",$272.88,Ontario College of Art and Design University,Professor,Florence Coordinator
115604,Universities,Bender,Daniel Eric,"$166,542.00",$433.68,University of Toronto,Professor of Historical and Cultural Studies,Director of Culinaria Research Centre
116990,Universities,Kingston,Paul,"$164,740.58",$260.28,University of Toronto,Professor of Political Science,Director of Centre For Critical Development S...
117503,Universities,Menou,Kristen,"$163,778.33",$520.20,University of Toronto,Professor of Physical and Environmental Sciences,Inaugural Director For Centre For Planetary S...


Unfortunately, 65 rows are affected. We need all values in the Calendar Year column to be 2016, meaning that we need to get rid of the current values somehow. It seems as though they are job titles. For example, the third row has value "Deputy Minister" in the Calendar Year column. The corresponding job title is "Housing", which surely cannot be a job title. On the other hand, we see that the row below it has job title "Director". But, the value in the Calendar Year column is "seconded to Stevenson Memorial Hospital as CEO".

From the [2015 salary disclosure page](https://www.ontario.ca/page/public-sector-salary-disclosure-2015-all-sectors-and-seconded-employees):
>Someone who is “seconded” has a job in a public sector organization (other than an Ontario government ministry), but currently works within a government ministry. The organization pays the person’s salary and benefits and the ministry reimburses the organization.

Since the dataframes at least have the same column names at this point, we will concatenate them to make our job easier for this section.

In [18]:
pss_comb = pd.concat([pss[year] for year in YEARS]).copy()

Let's take a closer look at a few people's job titles and corresponding calendar year values in attempt to find a solution to this problem.

In [19]:
pss_comb[pss_comb['Last Name'].str.contains('Fagan', case=False) & pss_comb['First Name'].str.contains('Thomas', case=False)]

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
5752,Ontario Public Service,FAGAN,THOMAS F,"$104,654.07",$285.40,Attorney General,"Director, Court Operations",2000
7247,Ontario Public Service,FAGAN,THOMAS,"$110,957.16",$254.85,Attorney General,"Director, Court Operations",2001
8590,Ontario Public Service,FAGAN,THOMAS,"$134,556.84",$223.52,Attorney General,"Director, Court Operations",2002
2234,Government of Ontario : Ministries,FAGAN,THOMAS,"$130,123.23",$231.59,Attorney General,"Director, Court Operations",2003
2288,Government of Ontario : Ministries,FAGAN,THOMAS,"$138,193.89",$240.95,Attorney General,"Director, Court Operations",2004
2854,Government of Ontario : Ministries,FAGAN,THOMAS,"$142,724.90",$235.49,Attorney General,"Director, Court Operations",2005
2586,Government of Ontario : Ministries,FAGAN,THOMAS,"$159,197.24",$268.40,Attorney General,"Director, Court Operations",2006
3647,Government of Ontario : Ministries,FAGAN,THOMAS,"$172,187.73",$301.74,Attorney General,"Director, Court Operations",2007
4959,Government of Ontario : Ministries,FAGAN,THOMAS,"$185,823.02",$332.66,Attorney General,"Director, Court Operations",2008
6283,Government of Ontario : Ministries,FAGAN,THOMAS,"$197,394.14",$326.25,Attorney General,"Director, Diversity Initiatives",2009


Fagan's job title in 2016 is very likely "Member", just as in subsequent years.

In [20]:
pss_comb[pss_comb['Last Name'].str.contains('leblanc', case=False) & pss_comb['First Name'].str.contains('Laurie', case=False)]

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
6715,Ontario Public Service,LEBLANC,LAURIE,"$100,117.22",$281.32,Municipal Affairs & Housing,"Director, Communications",2000
8760,Ontario Public Service,LEBLANC,LAURIE,"$108,753.38",$252.16,Municipal Affairs & Housing,"Director,Urb Aff & Stakeholders Rel Br.",2001
10468,Ontario Public Service,LEBLANC,LAURIE,"$110,957.01",$223.52,Municipal Affairs and Housing,"Director, Urban Aff. & Stakeholder Rel.",2002
4203,Government of Ontario : Ministries,LEBLANC,LAURIE,"$112,785.31",$207.09,Municipal Affairs & Housing,"Director, Urb Affrs & Stk Rel",2003
3080,Government of Ontario : Ministries,LEBLANC,LAURIE,"$124,170.53",$210.70,Cabinet Office,"ADM, Communications",2004
3676,Government of Ontario : Ministries,LEBLANC,LAURIE,"$139,155.05",$217.29,Cabinet Office,"ADM, Communications",2005
3514,Government of Ontario : Ministries,LEBLANC,LAURIE,"$157,447.13",$262.75,Cabinet Office,"Assistant Deputy Minister, Communications",2006
5661,Government of Ontario : Ministries,LEBLANC,LAURIE,"$167,344.55",$280.28,Community Safety & Correctional Services,"Assistant Deputy Minister, Policy & Strategic ...",2007
7314,Government of Ontario : Ministries,LEBLANC,LAURIE,"$176,426.00",$294.60,Community Safety & Correctional Services,"Assistant Deputy Minister, Policy & Strategic ...",2008
12476,Government of Ontario : Ministries,LEBLANC,LAURIE,"$178,463.46",$285.32,"Training, Colleges & Universities","Assistant Deputy Minister, Employment and Trai...",2009


It is clear that in 2016, Laurie Leblanc's job title should be "Deputy Minister". So, we could replace "Housing" with "Deputy Minister". But, the problem is that this is not a general solution. For example, is Dora Cavallo-Medved's (in the wrong_year dataframe, third row from the bottom) job title "Sessional Lecturer I" or "Course Developer"?

In [21]:
pss_comb[pss_comb['Last Name'].str.contains('levac', case=False) & pss_comb['First Name'].str.contains('jody', case=False)]

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
10105,Hospitals and Boards of Public Health,LEVAC,JODY,"$103,611.97","$6,443.50",Southlake Regional Health Centre,"Director, Mental Health",2007
13496,Hospitals and Boards of Public Health,LEVAC,JODY,"$115,024.89","$5,869.93",Southlake Regional Health Centre,"Director, Mental Health",2008
16432,Hospitals and Boards of Public Health,LEVAC,JODY,"$116,763.88","$6,339.81",Southlake Regional Health Centre,"Director, Mental Health",2009
68542,Hospitals and Boards of Public Health,LEVAC,JODY,"$120,334.86","$4,690.41",Southlake Regional Health Centre,Manager,2010
20999,Hospitals and Boards of Public Health,LEVAC,JODY,"$122,959.69","$6,649.28",Southlake Regional Health Centre / Centre régi...,Director/Directeur,2011
22835,Hospitals and Boards of Public Health,LEVAC,JODY,127370.77,13498.25,Southlake Regional Health Centre,Director,2012
23872,Hospitals and Boards of Public Health,LEVAC,JODY,132595.35,501.02,Southlake Regional Health Centre,Director / Directeur,2013
100867,Hospitals and Boards of Public Health,Levac,Jody J.,"$176,593.90",$632.04,Southlake Regional Health Centre,Director and Chief Executive Officer Stevenson...,2014
103961,Hospitals & Boards of Public Health,Levac,Jody J.,"$169,940.16",$632.04,Southlake Regional Health Centre,Director; seconded to Stevenson Memorial Hospi...,2015
32225,Hospitals and Boards of Public Health,Levac,Jody J.,"$169,367.20",$622.35,Southlake Regional Health Centre,Director,seconded to Stevenson Memorial Hospital as CEO


In 2011, 2013, and 2018, the job titles are of the form "English / French", unlike the other years. Even worse, in 2011, both the employer and job title are of the form "English / French" but in 2013, only the job title is. Even the name columns are inconsistent: from 2007 to 2013, the first and last names are in upper case, unlike 2014 to 2019. From 2014 to 2017, the first name column has an initial "J.", which is not consistent with the rest of the years.

This one gives us an idea. We could append the string in the Calendar Year column to the job title column, separated by a semicolon, as in 2015.

In [22]:
pss[2016].loc[pss[2016]['Calendar Year'] != '2016', 'Job Title'] = wrong_year['Job Title'].str.cat(wrong_year['Calendar Year'], sep='; ')
pss[2016].loc[pss[2016]['Calendar Year'] != '2016', 'Calendar Year'] = '2016'
pss[2016]['Calendar Year'] = pss[2016]['Calendar Year'].astype('int')
pss[2016].dtypes

Sector              object
Last Name           object
First Name          object
Salary Paid         object
Taxable Benefits    object
Employer            object
Job Title           object
Calendar Year        int64
dtype: object

Had we dropped the Calendar Year column before examining it closely, we would have lost job title information for 65 rows in the 2016 dataframe. Therefore, it is always a good idea to examine a column before dropping it when cleaning data.

Before moving on, we will take a look at the Calendar Year values in each dataframe to make sure that there are no surprises.

In [23]:
for year in YEARS:
    if pss[year]['Calendar Year'].nunique() > 1:
        print(year, pss[year]['Calendar Year'].unique(), sep='\n')

2015
[2015 2016 2017 2018]


It looks like the Calendar Year column in the 2015 dataframe has values other than 2015. Another important lesson: if we had assumed that there were no mistakes in the Calendar Year column, we could make some serious errors. For example, if we decided to concatenate the dataframes and perform a groupby operation on the Calendar Year column, then several rows would be incorrectly grouped into 2016, 2017, and 2018.

In [24]:
wrong_year_2015 = pss[2015][pss[2015]['Calendar Year'] != 2015]
wrong_year_2015

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
76671,Universities,Webb,Paul,"$105,001.75",$622.01,King's University College,Professor,2016
111692,Other Public Sector Employers,Yue,Jenny,"$114,158.53",$131.94,Colleges Ontario,"Senior Manager, Finance And Administration",2016
113433,Other Public Sector Employers,Macdonald,Christine,"$100,789.02",$-,Mitacs Inc.,"Director, Business Development & Team Lead Ont...",2016
113434,Other Public Sector Employers,Parlee,Forrest,"$108,709.89",$-,Mitacs Inc.,"Director, Partnerships",2017
115175,Other Public Sector Employers,Udovic,Natasha,"$129,807.63",$867.92,"Toronto Festival of Arts, Culture and Creativity","Senior Director, Corporate Partnerships",2016
115176,Other Public Sector Employers,Wagner,John Clyde,"$168,230.79",$867.92,"Toronto Festival of Arts, Culture and Creativity",Executive Producer,2017
115177,Other Public Sector Employers,Weisbrodt,Jorn,"$207,692.37","$1,272.92","Toronto Festival of Arts, Culture and Creativity",Artistic Director,2018
115248,Other Public Sector Employers,Tanos,Antonia,"$102,222.20","$3,664.48",Toronto Organizing Committee for the 2015 Pan ...,Senior Project Management Analyst / Analyste p...,2016


Only a few of the rows are affected. Since this data was released in 2016 for the calendar year 2015, the above values don't make sense. We will correct these now.

In [25]:
pss[2015].loc[wrong_year_2015.index, 'Calendar Year'] = 2015

Now that the Calendar Year column has the same data type across all dataframes, we will move on to examine the Salary Paid and Taxable Benefits columns. We already saw that we need to remove dollar signs and commas. But, is that everything? We will check for other non-numeric characters. In the two code cells below, we search for any non-numeric characters excluding dollar signs, commas, periods, and spaces.

In [26]:
pss_comb = pd.concat([pss[year] for year in YEARS]).copy().reset_index(drop=True)

salary_nonnum = pss_comb['Salary Paid'].str.extractall('([^$\d.,\s])').drop_duplicates()
salary_nonnum

,,0
,match,


In [27]:
tax_ben_nonnum = pss_comb['Taxable Benefits'].str.extractall('([^$\d.,\s])').drop_duplicates()
tax_ben_nonnum

,,0
,match,
780155,0,-


It looks like we have dashes in the Taxable Benefits column. We will need to remove these before converting to this column to the `float` data type. Before we do so, we should take a look at how the dash appears in the data.

In [28]:
idx = tax_ben_nonnum.reset_index(level='match', drop=True).index
pss_comb.loc[idx, 'Taxable Benefits'].unique()

array(['$-'], dtype=object)

The dash only appears in the form "$-".

Once we have replaced all dollar signs and commas, we will replace all dashes with `np.nan` since these are missing values.

In [29]:
for year in YEARS:
    pss[year]['Salary Paid'] = (pss[year]['Salary Paid']
                                .replace('[$,]', '', regex=True)
                                .replace('-', np.nan)
                                .astype('float')
                               )
    pss[year]['Taxable Benefits'] = (pss[year]['Taxable Benefits']
                                     .replace('[$,]', '', regex=True)
                                     .replace('-', np.nan)
                                     .astype('float')
                                    )

Finally, we convert the columns that have data type `object` to `string`. We may do so by calling `convert_dtypes()` on each dataframe. This will convert the columns to the best possible data types. The [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html#dtypes) explains why this is a good idea:

> Pandas has two ways to store strings.
> 1. object dtype, which can hold any Python object, including strings.
> 1. StringDtype, which is dedicated to strings.
> 
> Generally, we recommend using StringDtype.
> Finally, arbitrary objects may be stored using the object dtype, but should be avoided to the extent possible (for performance and interoperability with other libraries and methods).

In [30]:
for year in YEARS:
    pss[year] = pss[year].convert_dtypes()
    print(year, pss[year].dtypes, sep='\n', end='\n\n')

1996
Sector               string
Last Name            string
First Name           string
Salary Paid         Float64
Taxable Benefits    Float64
Employer             string
Job Title            string
Calendar Year         Int64
dtype: object

1997
Sector               string
Last Name            string
First Name           string
Salary Paid         Float64
Taxable Benefits    Float64
Employer             string
Job Title            string
Calendar Year         Int64
dtype: object

1998
Sector               string
Last Name            string
First Name           string
Salary Paid         Float64
Taxable Benefits    Float64
Employer             string
Job Title            string
Calendar Year         Int64
dtype: object

1999
Sector               string
Last Name            string
First Name           string
Salary Paid         Float64
Taxable Benefits    Float64
Employer             string
Job Title            string
Calendar Year         Int64
dtype: object

2000
Sector             

Each column now has the desired data type. Since we are writing this data to a CSV, we will need to convert the data types again when we explore this data in a different notebook. So, why convert data types here? We ran into several issues while attempting to do so. The idea is to take care of those issues here instead of in an analysis notebook.

<a id="missing-values"></a>
[Return to table of contents](#table-of-contents)

# Missing values

In [31]:
for year in YEARS:
    if pss[year].isna().sum().sum() != 0:
        print(year, pss[year].isna().sum(), sep='\n', end='\n\n')

1996
Sector              0
Last Name           0
First Name          1
Salary Paid         0
Taxable Benefits    0
Employer            0
Job Title           0
Calendar Year       0
dtype: int64

1997
Sector              0
Last Name           0
First Name          0
Salary Paid         0
Taxable Benefits    0
Employer            0
Job Title           1
Calendar Year       0
dtype: int64

1998
Sector              0
Last Name           0
First Name          1
Salary Paid         0
Taxable Benefits    0
Employer            0
Job Title           0
Calendar Year       0
dtype: int64

2013
Sector              0
Last Name           0
First Name          1
Salary Paid         0
Taxable Benefits    0
Employer            0
Job Title           0
Calendar Year       0
dtype: int64

2015
Sector                 0
Last Name              0
First Name             0
Salary Paid            0
Taxable Benefits    6980
Employer               0
Job Title              0
Calendar Year          0
dtype: int64

2

Apart from the 6980 in the 2015 dataframe, there are not many missing values. Let's first find out why there are so many missing values in the Taxable Benefits column in 2015. Maybe those missing values are supposed to be zeros.

In [32]:
pss[2015]['Taxable Benefits'].eq(0).sum()

0

It seems as if every employee received some amount of taxable benefits in 2015. This seems unlikely, but we could take a look at the data from other years and compare.

In [33]:
pss_comb = (pd.concat([pss[year] for year in YEARS])
            .copy()
            .reset_index(drop=True)
           )
no_tax_ben = (pss_comb
              .loc[pss_comb['Taxable Benefits'].eq(0), 'Calendar Year']
              .value_counts()
              .reindex(list(YEARS), fill_value=0)
              .to_frame()
              .reset_index()
              .rename(columns={'index': 'Calendar Year', 'Calendar Year': 'Number of employees'})
             )

In [34]:
fig = px.scatter(no_tax_ben, x='Calendar Year', y='Number of employees')
fig.update_traces(mode='lines+markers',
                  hovertemplate=
                  '<b>%{x}</b><br>'+
                  'Number of employees: <b>%{y}</b>'
                 )
fig.update_layout(title='Number of employees that did not receive taxable benefits by calendar year',
                  xaxis_title='Calendar Year',
                  yaxis_title="Number of employees",
                  yaxis_tickformat=',',
                  hoverlabel_bgcolor="white",
                  hoverlabel_font_size=14,
                  hovermode="x",
                  yaxis_zerolinecolor='grey',
                  yaxis_zerolinewidth=1
                 )
fig.show(config=CONFIG)

Based on the above plot, we can reasonably assume that the missing values for the 2015 Taxable Benefits column must be zeros. We will fill in the missing values and then plot the data again.

In [35]:
pss[2015]['Taxable Benefits'].fillna(0.0, inplace=True)
no_tax_ben.loc[no_tax_ben['Calendar Year'].eq(2015), 'Number of employees'] = pss[2015]['Taxable Benefits'].eq(0).sum()

In [36]:
fig = px.scatter(no_tax_ben, x='Calendar Year', y='Number of employees')
fig.update_traces(mode='lines+markers',
                  hovertemplate=
                  '<b>%{x}</b><br>'+
                  'Number of employees: <b>%{y}</b>'
                 )
fig.update_layout(title='Number of employees that did not receive taxable benefits by calendar year',
                  xaxis_title='Calendar Year',
                  yaxis_title="Number of employees",
                  yaxis_tickformat=',',
                  hoverlabel_bgcolor="white",
                  hoverlabel_font_size=14,
                  hovermode="x",
                  yaxis_zerolinecolor='grey',
                  yaxis_zerolinewidth=1
                 )
fig.show(config=CONFIG)

The plot makes more sense now. The number of employees that did not receive taxable benefits in 2015 is between the corresponding values for 2014 and 2016.

The 2016 data has one missing value in the Taxable Benefits column.

In [37]:
null_2016 = pss[2016][pss[2016]['Taxable Benefits'].isna()]
null_2016

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
84270,Other Public Sector Employers,Malenfant,James,152545.47,<NA>,Independent Electricity System Operator,Senior Analyst - Compliance and Dispute Resolu...,2016


We will look for "James Malenfant" in other years in hopes of being able to impute this value.

In [38]:
pss_comb[pss_comb['Last Name'].eq('Malenfant') & pss_comb['First Name'].eq('James')]

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
892807,Other Public Sector Employers,Malenfant,James,107325.45,<NA>,Independent Electricity System Operator,Senior Analyst - Compliance & Dispute Resoluti...,2015
980270,Other Public Sector Employers,Malenfant,James,152545.47,<NA>,Independent Electricity System Operator,Senior Analyst - Compliance and Dispute Resolu...,2016
1114900,Other Public Sector Employers,Malenfant,James,153909.53,0.0,Independent Electricity System Operator / Opér...,"Senior Analyst, Compliance and Dispute Resolut...",2017
1256693,Other Public Sector Employers,Malenfant,James,155282.66,0.0,Independent Electricity System Operator,Senior Analyst - Compliance and Dispute Resolu...,2018
1417573,Other Public Sector Employers,Malenfant,James,158596.56,0.0,Independent Electricity System Operator,Senior Analyst - Compliance and Dispute Resolu...,2019


Since James Malenfant does not usually receive taxable benefits, it is reasonable to assume that he didn't receive taxable benefits in 2016 either.

In [39]:
pss[2016].loc[null_2016.index, 'Taxable Benefits'] = 0.0

The 2013 data has one missing value in the First Name column.

In [40]:
null_2013 = pss[2013][pss[2013]['First Name'].isna()]
null_2013

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
4615,Crown Agencies,LI,<NA>,108421.82,47.18,eHealth Ontario,Privacy and Security Architect,2013


In [41]:
pss_comb[pss_comb['Last Name'].str.contains('^li$', case=False) & pss_comb['Employer'].str.contains('eHealth') & pss_comb['Job Title'].str.contains('privacy', case=False)]

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
575124,Crown Agencies,LI,<NA>,108421.82,47.18,eHealth Ontario,Privacy and Security Architect,2013
683346,Crown Agencies,Li,Na,106867.01,42.12,eHealth Ontario,Privacy and Security Architect / Architecte de...,2014


We can safely assume that Li's first name is Na based on the record from 2014. It looks like the first name "NA" was marked as a missing value in the 2013 data. The first name "Na" in the 2014 data was not marked as a missing value. From the `pandas.read_csv` [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html):

> **na_values: scalar, str, list-like, or dict, optional**   
> Additional strings to recognize as NA/NaN. If dict passed, specific per-column NA values. By default the following values are interpreted as NaN: ‘’, ‘#N/A’, ‘#N/A N/A’, ‘#NA’, ‘-1.#IND’, ‘-1.#QNAN’, ‘-NaN’, ‘-nan’, ‘1.#IND’, ‘1.#QNAN’, ‘<NA>’, ‘N/A’, ‘NA’, ‘NULL’, ‘NaN’, ‘n/a’, ‘nan’, ‘null’.
    
This means that LI's first name "NA" in the 2013 data was likely never really a missing value to begin with. Let's find out what the original first name value was in 2013. We can do so by passing the argument `False` to the `na_filter` parameter when calling `pd.read_csv`.

In [42]:
pss2013 = pd.read_csv(path/'pssd-en-2013.csv', na_filter=False)
pss2013[pss2013['Last Name'].str.contains('^li$', case=False) & pss2013['Employer'].str.contains('eHealth') & pss2013['Job Title'].str.contains('privacy', case=False)]

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
4615,Crown Agencies,LI,NA,108421.82,47.18,eHealth Ontario,Privacy and Security Architect,2013


Indeed we see that the first name was originally "NA", but was marked as a missing value. We will set the missing first name to "NA" instead of "Na" in order to be consistent, since names in the 2013 data seem to be in upper case only.

In [43]:
pss[2013].loc[null_2013.index, 'First Name'] = 'NA'

The 1998 data has one missing value in the first name column.

In [44]:
null_1998 = pss[1998][pss[1998]['First Name'].isna()]
null_1998

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
595,Crown Agencies,DONNELLY,<NA>,100813.0,21526.0,Ontario Hydro,"Mgr, Labour Reltns, IMO/CMO/N-",1998


In [45]:
pss_comb[pss_comb['Last Name'].str.contains('donnelly', case=False) & pss_comb['Employer'].str.contains('hydro', case=False)]

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
10474,Crown Agencies,DONNELLY,<NA>,100813.0,21526.0,Ontario Hydro,"Mgr, Labour Reltns, IMO/CMO/N-",1998
17761,Hydro One & Ontario Power Generation,DONNELLY,N.,113935.62,1098.59,Hydro One,Labour Relations Manager,1999


Based on the record from 1999, we can safely assume that Donnelly's first initial is "N". However, just as we saw above, it may be the case that `pd.read_csv` marked Donnelly's first name as a missing value.

In [46]:
pss1998 = pd.read_csv(path/'en-1998-pssd.csv', encoding='latin1', na_filter=False)
pss1998[pss1998['Last Name'].str.contains('donnelly', case=False) & pss1998['Employer'].str.contains('hydro', case=False)]

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
595,Crown Agencies,DONNELLY,NA,"$100,813.00","$21,526.00",Ontario Hydro,"Mgr, Labour Reltns, IMO/CMO/N-",1998


Just like before, a first name of "NA" was marked as a missing value. Donnelly's first name is probably not "NA" though. It's more likely that these are initials. It's not uncommon in the 1998 data to have 2 letter initials in the First Name column.

In [47]:
pss[1998][pss[1998]['First Name'].str.len().eq(2)].head()

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
266,Crown Agencies,ACCHIONE,PN,107527.0,10377.0,Ontario Hydro,"Manager, Programming",1998
269,Crown Agencies,ADAMS,RJ,107327.0,28414.0,Ontario Hydro,"Manager, Operations Production",1998
273,Crown Agencies,AITKEN,RB,109803.0,463.0,Ontario Hydro,Shift Maintenance Supervisor-Mechanical,1998
281,Crown Agencies,ALLEN,MI,100884.0,266.0,Ontario Hydro,Intermediate Mail and Services Clerk,1998
285,Crown Agencies,ANDERSON,WA,100561.0,330.0,Ontario Hydro,Inspection and Maintenance Technician I,1998


Therefore, we'll set the missing first name to "NA", as it was originally.

In [48]:
pss[1998].loc[null_1998.index, 'First Name'] = 'NA'

The 1997 data has one missing value in the Job Title column.

In [49]:
null_1997 = pss[1997][pss[1997]['Job Title'].isna()]
null_1997

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
963,Crown Agencies,Walker,G J,104932.87,403.32,Ontario Hydro,<NA>,1997


In [50]:
pss_comb[pss_comb['Last Name'].str.contains('walker', case=False) & pss_comb['Employer'].str.contains('ontario hydro', case=False)]

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
2302,CROWN AGENCIES,WALKER,J,103850.0,416.0,Ontario Hydro,AUTHORIZED NUCLEAR OPERATOR,1996
2303,CROWN AGENCIES,WALKER,J,112519.0,536.0,Ontario Hydro,SHIFT OPERATING SUPERVISOR (AUTHORIZED),1996
2304,CROWN AGENCIES,WALKER,W,136763.0,712.0,Ontario Hydro,"SERVICE MGR, INFORMATION TECHNOLOGY SRVC",1996
5462,Crown Agencies,Walker,Colleen M,160369.87,1267.71,Ontario Hydro,EXECUTIVE ASSISTANT,1997
5463,Crown Agencies,Walker,D G,124642.07,418.83,Ontario Hydro,SENIOR ENGINEER/SCIENTIST/TECH OFFICER,1997
5464,Crown Agencies,Walker,G J,104932.87,403.32,Ontario Hydro,<NA>,1997
5465,Crown Agencies,Walker,J H,113798.7,487.05,Ontario Hydro,SHIFT OPERATING SUPERVISOR (AUTHORIZED),1997
5466,Crown Agencies,Walker,Wayne A,123669.6,2011.86,Ontario Hydro,"SERVICE MGR, INFORMATION TECHNOLOGY SRV",1997
11457,Crown Agencies,WALKER,G,113573.0,489.0,Ontario Hydro,Maintenance Superintendent,1998
11458,Crown Agencies,WALKER,G,126936.0,480.0,Ontario Hydro,Senior Engineer/Scientist/Tech Officer,1998


Since they have the same job title, "Walker, D G" from 1997 and "Walker, G" from 1998 seem to be the same person. The other "Walker, G" has the job title "Maintenance Superintendent" in 1998, which is one year after "Walker, G J" in 1997. My best guess then, is to say that "Walker, G J" has job title "Maintenance Superintendent". This [external page](https://www.ontariosunshinelist.com/people/fqpyng) seems to support my claim.

In [51]:
pss[1997].loc[null_1997.index, 'Job Title'] = 'Maintenance Superintendent'

The 1996 data has one missing value in the First Name column.

In [52]:
null_1996 = pss[1996][pss[1996]['First Name'].isna()]
null_1996

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
1714,MUNICIPALITIES,Yearwood,<NA>,116371.0,978.0,Municipality of Metropolitan Toronto,Deputy Commissioner of Corp. Serv.,1996


In [53]:
pss_comb[pss_comb['Last Name'].str.contains('yearwood', case=False) & pss_comb['Calendar Year'].le(2010)]

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
1714,MUNICIPALITIES,Yearwood,<NA>,116371.0,978.0,Municipality of Metropolitan Toronto,Deputy Commissioner of Corp. Serv.,1996
130927,Universities,YEARWOOD,LOUISE,109492.62,390.14,Ryerson University,"Director, Development, C&D",2005
355991,Municipalities and Services,YEARWOOD,MICHELLE,102838.21,405.75,City of Brampton,"Senior Advisor, Performance Management",2010


This is not helpful. Let's find out what the original value of the first name was.

In [54]:
pss1996 = pd.read_csv(path/'en-1996-pssd.csv', encoding='latin1', na_filter=False)
pss1996[pss1996['Last Name'].str.contains('yearwood', case=False)]

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year,Unnamed: 8
1714,MUNICIPALITIES,Yearwood,NA,"$116,371.00",$978.00,Municipality of Metropolitan Toronto,Deputy Commissioner of Corp. Serv.,1996,


Again, a first name of "NA" was interpreted to be a missing value by the `pd.read_csv` function. This is the last missing value. If we had many missing values, we would have to deal with them in a less cumbersome way. For example, we could specify that we only want empty strings to be interpreted as missing values when reading in the data with `pd.read_csv`.

Just like before, we'll keep the original first name of "NA", since it's not uncommon to have 2 letter initials in the 1996 data.

In [55]:
pss[1996][pss[1996]['First Name'].str.len().eq(2)].head()

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
605,ONTARIO PUBLIC SERVICE,WEBSTER,AR,123825.0,0.0,Attorney General,"Provincial Judge, OCJ",1996
820,MUNICIPALITIES,Baxter,SK,106526.0,374.0,City of Toronto,City Clerk,1996
851,MUNICIPALITIES,Wichmann,WM,105603.0,33.0,City of Toronto,"Director, Engineering & Surveys",1996
892,MUNICIPALITIES,Lobo,J.,116588.0,691.0,Corporation of the City of Hamilton,Commissioner of Public Works & Traffic Serv.,1996
1072,HOSPITALS,Arsenault,D.,146827.0,8077.0,Hamilton Health Sciences Corporation,Vice-President,1996


In [56]:
pss[1996].loc[null_1996.index, 'First Name'] = 'NA'

<a id="concat-data"></a>
[Return to table of contents](#table-of-contents)

# Concatenating the dataframes

In [57]:
pss_comb = pd.concat([pss[year] for year in YEARS]).copy()

The data spans from 1996 to 2019 and as we saw throughout this notebook, there were many inconsistencies. Here is just one of many examples:

In [58]:
pss_comb[pss_comb['Last Name'].str.contains('malenfant', case=False) & pss_comb['First Name'].str.contains('andrew', case=False)]

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
33002,Municipalities and Services,MALENFANT,ANDREW DEREK,110851.7,324.09,City of Toronto - Police Service,Police Constable,2010
33427,Municipalities and Services,MALENFANT,ANDREW DEREK,115988.74,751.24,City of Toronto - Police Service,Plainclothes Police Constable,2011
48487,Municipalities and Services,MALENFANT,ANDREW DEREK,116828.38,686.42,City of Toronto - Police Service,Plainclothes Police Constable,2012
54019,Municipalities and Services,MALENFANT,ANDREW DEREK,108888.82,759.93,City of Toronto - Police Service,Police Constable,2013
46296,Municipalities and Services,Malenfant,Andrew Derek,117105.33,754.78,City of Toronto - Police Service,Police Constable,2014
43728,Municipalities & Services,Malenfant,Andrew Derek,137741.05,818.86,City of Toronto - Police Service,Police Constable,2015
56955,Municipalities and Services,Malenfant,Andrew Derek,131296.57,981.89,City of Toronto - Police Service,Police Constable,2016
69543,Municipalities and Services,Malenfant,Andrew Derek,131755.36,1000.74,City of Toronto - Police Service,Police Constable,2017
72618,Municipalities and Services,Malenfant,Andrew Derek,135177.3,1008.17,City of Toronto - Police Service,Police Constable,2018
80388,Municipalities & Services,Malenfant,Andrew,146555.36,1023.96,City Of Toronto - Police Service,Police Constable,2019


I think this is a good example because it displays several inconsistencies. It is clear that these records all belong to the same person. In the Sector column, some rows use the ampersand (&) instead of the word "and". In the Last Name and First Name columns, some values are in upper case while others are not. Also, in 2019, the first name is just "Andrew" instead of "Andrew Derek".

It would be nice if each person were given some sort of unique id. This way, even is a person's name changes, or if we see something like the example above, we would be able to easily tell whether two people on the list are the same person.

It is much easier to work with one dataframe as opposed to 24 and the calendar year column can always be used to separate the data again if needed. Another reason one CSV file is preferred over 24 is that file descriptions and column descriptions in the resulting dataset will only need to be specified one time instead of 24 times. I will also add a note in the dataset description about the inconsistency of the data across calendar years.

<a id="write-to-csv"></a>
[Return to table of contents](#table-of-contents)

# Write to CSV

In [59]:
pss_comb

,Sector,Last Name,First Name,Salary Paid,Taxable Benefits,Employer,Job Title,Calendar Year
0,OTHER PUBLIC SECTOR EMPLOYERS,Kendall,Perry,194890.4,711.24,Addiction Research Foundation,President & CEO,1996
1,OTHER PUBLIC SECTOR EMPLOYERS,Rehm,Juergen,115603.62,403.41,Addiction Research Foundation,"Dir., Soc. Eval. Research & Act. Dir., Clin. R...",1996
2,OTHER PUBLIC SECTOR EMPLOYERS,Room,Robin,149434.48,512.58,Addiction Research Foundation,"V.P., Research & Coordinator, Intern. Programs",1996
3,ONTARIO PUBLIC SERVICE,KNOX,KEN W,109382.92,4921.68,"Agriculture,Food and Rural Affairs",Deputy Minister,1996
4,HOSPITALS,Cliff,Bruce,110309.0,3157.0,Ajax and Pickering General Hospital,President & CEO,1996
...,...,...,...,...,...,...,...,...
166972,Universities,Zoidl,Georg R,226479.06,1104.16,York University,Professor,2019
166973,Universities,Zryd,Michael,171004.6,910.84,York University,Associate Professor,2019
166974,Universities,Zumbansen,Peer C,193639.44,1104.16,York University,Professor,2019
166975,Universities,Zwick,Detlev,256491.58,1104.16,York University,Associate Dean / Associate Professor,2019


We sort the data as a final step before writing the data to a CSV file.

In [60]:
pss_comb.sort_values(['Calendar Year', 'Sector', 'Employer', 'Last Name', 'First Name']).to_csv('pssd.csv', index=False)